In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

path = "D:\\Coding\Machine_Learning\Recommendation_System\\"

In [2]:
movies_item = ['MovieID','Title','Genres']

users_item = ['UserID','Gender','Age','Occupation','Zip-code']

ratings_item = ['UserID','MovieID','Rating','Timestamp']

genres = [ 'Action' ,'Adventure','Animation','Children\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir'
,'Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.DataFrame()
ratings = pd.DataFrame()
users = pd.DataFrame()

In [3]:
folder = 'ml-25M'
from_loc = f'{path}\\prepairing_Data\\{folder}\\'
# movies = pd.read_csv(f'{from_loc}\\movies.dat', sep='::', header=None, names=movies_item, encoding='latin1')
# ratings = pd.read_csv(f'{from_loc}\\ratings.dat', sep='::', header=None, names=ratings_item, encoding='latin1')
# users = pd.read_csv(f'{from_loc}\\users.dat', sep='::', header=None, names=users_item, encoding='latin1')

In [4]:
movies  = pd.read_csv(f'{from_loc}\\movies.csv', sep=',', header=None, names=movies_item, engine='python', skiprows=1)
ratings = pd.read_csv(f'{from_loc}\\ratings.csv', sep=',', header=None, names=ratings_item, engine='python', skiprows=1)
# users   = pd.read_csv(f'{from_loc}\\users.csv', sep=',', header=None, names=users_item, engine='python', skiprows=1)

In [5]:
if 'Years' not in movies.columns:
    # แยกปีออกจาก Title และ แปลง Genres เป็นรายการ
    movies['Years'] = movies['Title'].str.extract(r'\((\d{4})\)')[0]
    movies['Years'] = pd.to_numeric(movies['Years'])

    # movies['Genres'] = movies['Genres'].str.split('|')

    # ลบวงเล็บและปีออกจาก Title
    movies['Title'] = movies['Title'].str.split('(').str[0]
    
    ratings = ratings.rename({'Rating':'userRating'}, axis='columns')
    ratings['Years'] = pd.to_datetime(ratings['Timestamp'], unit="s").dt.year
    # ratings['Timestamp'] = pd.to_datetime(ratings['Timestamp'], unit="s").dt.date
    


In [6]:
from faker import Faker

fake = Faker()
userIDs = ratings["UserID"].unique()

dic_users = {"UserID":[], "FirstName":[], "LastName":[], "Email":[], "Password":[]}

# สร้างข้อมูลผู้ใช้แบบสุ่ม
for i in userIDs:  
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = fake.email()
    # password = fake.password()
    password = "12345678"
    
    dic_users["UserID"].append(int(i))
    dic_users["FirstName"].append(first_name)
    dic_users["LastName"].append(last_name)
    dic_users["Email"].append(email)
    dic_users["Password"].append(password)

users = pd.DataFrame(dic_users, columns=["UserID", "FirstName", "LastName", "Email", "Password"])

In [7]:
def display_pd(df_clean):
    display(df_clean.head(5), df_clean.shape)

In [8]:
def check_Years(df_clean):   
    movie_by_years_count = df_clean.groupby('Years').count()['MovieID'].sort_values(ascending=True).reset_index()
    movie_by_years_count.rename(columns={'MovieID':'No-of-Years'}, inplace=True)

    return movie_by_years_count.describe()

In [9]:
movies_ratings_count = ratings[['MovieID', 'UserID']].groupby('MovieID').count()['UserID'].reset_index()
movies_ratings_count.rename(columns={'UserID':'Number-of-Movie-Ratings'}, inplace=True)

movies_ratings_count.describe()

,MovieID,Number-of-Movie-Ratings
count,59047.000000,59047.000000
mean,120692.748065,423.393144
std,64452.508194,2477.885821
min,1.000000,1.000000
25%,78277.000000,2.000000
50%,136782.000000,6.000000
75%,173812.000000,36.000000
max,209171.000000,81491.000000


มีการดูหนังมากกว่า 1000 ครั้ง

In [10]:
df_movies_ratings = movies_ratings_count[movies_ratings_count['Number-of-Movie-Ratings'] >= 1000]
if 'Number-of-Movie-Ratings' in df_movies_ratings.columns :
    df_movies_ratings.drop(columns=['Number-of-Movie-Ratings'], inplace=True)
    
print("Before Count Movie:", movies.shape[0])
print("After  Count Movie:", df_movies_ratings.shape[0])

Before Count Movie: 62423
After  Count Movie: 3794


In [11]:
df_clean_movies = pd.merge(df_movies_ratings, movies, on='MovieID')

display(check_Years(df_clean_movies))
display_pd(df_clean_movies)

,Years,No-of-Years
count,98.000000,98.000000
mean,1970.438776,38.673469
std,28.539380,44.584398
min,1920.000000,1.000000
25%,1946.250000,5.250000
50%,1970.500000,14.500000
75%,1994.750000,62.750000
max,2019.000000,164.000000


,MovieID,Title,Genres,Years
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


(3794, 4)

In [23]:
df_movie_ratings = pd.merge(ratings, df_clean_movies['MovieID'], on='MovieID')
# display(filtered_ratings.head(5), filtered_ratings.shape)

Rating ต้องมากกว่า 10 ปีย้อนหลัง

เพราะ ในช่วงเวลา 10 ปี อาจจะสามารถเป็นนิสัยในการดูภาพยนตร์ของผู้ใช้งานได้เลย

In [31]:
from datetime import datetime

year_now = datetime.now().year # 2023

df_ratings_years = df_movie_ratings[year_now - df_movie_ratings['Years'] < 10]

display_pd(df_ratings_years)

,UserID,MovieID,userRating,Timestamp,Years
1,3,296,5.0,1439474476,2015
2,4,296,4.0,1573938898,2019
9,14,296,5.0,1506208897,2017
12,19,296,3.5,1466674637,2016
19,33,296,4.5,1552804447,2019


(6790572, 5)

Rating ล่าสุด ต้องมากกว่า 5 ปีย้อนหลังนับจากปัจจุบัน

เหมือนกับเช็ค ว่าที่ผู้ใช้งานที่Loginครั้งล่าสุดเมื่อ 5 ปีก่อน ข้อมูลน่าจะเก่าเกินไป และผู้ใช้อาจจะไม่เข้ามาใช้งานอีกแล้วก็ได้

In [32]:
df_last_ratings = df_ratings_years.groupby('UserID').max()['Years'].reset_index()
df_last_ratings.rename(columns={'Years':'Last-of-Years'}, inplace=True)
df_last_ratings.describe()

,UserID,Last-of-Years
count,48952.000000,48952.000000
mean,81249.899882,2016.801173
std,46993.879990,1.607167
min,3.000000,2014.000000
25%,40342.500000,2015.000000
50%,81139.500000,2017.000000
75%,122180.250000,2018.000000
max,162538.000000,2019.000000


In [46]:
df_clean_last_ratings = df_last_ratings[year_now - df_last_ratings['Last-of-Years'] < 5]
if 'Last-of-Years' in df_clean_last_ratings.columns :
    df_clean_last_ratings.drop(columns=['Last-of-Years'], inplace=True)


,UserID,Last-of-Years
count,10445.000000,10445.0
mean,80743.785065,2019.0
std,47059.205037,0.0
min,3.000000,2019.0
25%,39564.000000,2019.0
50%,80221.000000,2019.0
75%,121859.000000,2019.0
max,162536.000000,2019.0


In [47]:
filtered_ratings = pd.merge(df_ratings_years, df_clean_last_ratings, on='UserID')

if 'Timestamp' in filtered_ratings.columns :
    filtered_ratings.drop(columns=['Years'], inplace=True)
    filtered_ratings.drop(columns=['Timestamp'], inplace=True)

display_pd(filtered_ratings)

,UserID,MovieID,userRating
0,3,296,5.0
1,3,1217,5.0
2,3,1653,5.0
3,3,4308,3.0
4,3,5952,4.0


(2247560, 3)

จัดรูปแบบ Data ของ Movie และ User ตามที่มีในตาราง filtered_ratings

Movie

In [48]:
filtered_movies = pd.DataFrame(filtered_ratings['MovieID'].unique(), columns=['MovieID'])

df_movies = pd.merge(movies, filtered_movies, on='MovieID')
display_pd(df_movies)

,MovieID,Title,Genres,Years
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


(3794, 4)

User

In [53]:
# filtered_users = pd.DataFrame(filtered_ratings['UserID'].unique(), columns=['UserID'])

# df_users = pd.merge(users, filtered_users, on='UserID')

df_users = users.copy()
display_pd(df_users)

,UserID,FirstName,LastName,Email,Password
0,1,Jean,Holt,gina64@example.net,12345678
1,2,Mckenzie,Campbell,davislisa@example.net,12345678
2,3,Ariel,West,browneric@example.com,12345678
3,4,Timothy,Holmes,lynchjimmy@example.net,12345678
4,5,Javier,Lewis,raymondyoung@example.com,12345678


(162541, 5)

Rating

In [54]:
df_ratings = filtered_ratings.copy()

display_pd(df_ratings)

,UserID,MovieID,userRating
0,3,296,5.0
1,3,1217,5.0
2,3,1653,5.0
3,3,4308,3.0
4,3,5952,4.0


(2247560, 3)

Save File to database SQL

In [55]:
import sqlite3
import os

def saveDatabase(path, table, df):
    path += '\\' + table+'.db'

    # Check มี File นั้นหรือไม่
    key = os.path.exists(path)
    if key:
        os.remove(path)
    conn = sqlite3.connect(path)
    df.to_sql(table, conn, if_exists='replace', index=False)
    conn.close()
    print(f'Save: {table}')

In [56]:
path = r'D:\Coding\Machine_Learning\Recommendation_System\input'

saveDatabase(path, 'movies', df_movies)
saveDatabase(path, 'users', df_users)
saveDatabase(path, 'ratings', df_ratings)
# saveDatabase(path, 'filtered_ratings', filtered_ratings)

Save: movies
Save: users
Save: ratings
